In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import holidays


from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV, ParameterGrid, TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBRegressor

#   File paths
file_path_EDA = "C:/Users/regi4/OneDrive - Atlantic TU/Year 4/Predictive Analysis Docu's/THE MODEL/French bakery daily sales/Iterations Output/french_bakery_EDA_2.csv"
file_path_Feat_Engr = "C:/Users/regi4/OneDrive - Atlantic TU/Year 4/Predictive Analysis Docu's/THE MODEL/French bakery daily sales/Iterations Output/french_bakery_Ft_Eng_1.csv"

iteration_file_path = "C:/Users/regi4/OneDrive - Atlantic TU/Year 4/Predictive Analysis Docu's/THE MODEL/French bakery daily sales/Iterations Output"
results_file_path = "C:/Users/regi4/OneDrive - Atlantic TU/Year 4/Predictive Analysis Docu's/THE MODEL/French bakery daily sales/results"

#   Import the dataset
data = pd.read_csv(file_path_EDA)
bakery_df = pd.DataFrame(data)
bakery_df.head()

#   Function to assign the season based on the date variable
def get_season(date):
    if (date.month == 12 and date.day >= 21) or (date.month <= 3 and date.day <= 19):
        return "Winter"
    elif (date.month == 3 and date.day >= 20) or (date.month <= 6 and date.day <= 20):
        return "Spring"
    elif (date.month == 6 and date.day >= 21) or (date.month <= 9 and date.day <= 22):
        return "Summer"
    else:
        return "Autumn"
    
bakery_df =  bakery_df.drop("ticket_number", axis='columns')
bakery_df.head()

#   Agregate the Quantity sold per article and day
daily_data = data.groupby(['date', 'article']).agg({
    'Quantity':'sum',
    'unit_price': 'mean'
}).reset_index()
bakery_df = daily_data
bakery_df.head()

#   Convert the "date" variable to DateTime
bakery_df['date'] = pd.to_datetime(bakery_df['date'])

season_order = ["Winter", "Spring", "Summer", "Autumn"]

day_of_week_mapping = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6
}
season_mapping = {
    "Winter": 0,
    "Spring": 1,
    "Summer": 2,
    "Autumn": 3
}

#   New Date features
bakery_df['day_of_the_week'] = bakery_df['date'].dt.day_of_week
bakery_df['month'] = bakery_df['date'].dt.month
bakery_df['season'] = bakery_df['date'].apply(get_season)
bakery_df['season_numeric'] = bakery_df['season'].map(season_mapping)

#   Encoding Categorical Variables
label_encoder_article = LabelEncoder()
bakery_df['article_encoded'] = label_encoder_article.fit_transform(bakery_df['article'])

#   Introducing Interaction Features to the dataset
def create_interaction_features(bakery_df):
    data_int_ft = bakery_df

    data_int_ft['price_article_interaction'] = data_int_ft['unit_price'] * data_int_ft['article_encoded']
    data_int_ft['season_article_interaction'] = data_int_ft['season_numeric'] * data_int_ft['article_encoded']
    data_int_ft['weekly_avg_sales'] = data_int_ft.groupby(['article_encoded', 'day_of_the_week'])['Quantity'].transform('mean')  
    data_int_ft['monthly_avg_sales'] = data_int_ft.groupby(['article_encoded', 'month'])['Quantity'].transform('mean') 

    return data_int_ft

bakery_df = create_interaction_features(bakery_df)
bakery_df.to_csv(f'{iteration_file_path}/french_bakery_Daily_Sales.csv', index=False)
bakery_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28597 entries, 0 to 28596
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date                        28597 non-null  datetime64[ns]
 1   article                     28597 non-null  object        
 2   Quantity                    28597 non-null  float64       
 3   unit_price                  28597 non-null  float64       
 4   day_of_the_week             28597 non-null  int32         
 5   month                       28597 non-null  int32         
 6   season                      28597 non-null  object        
 7   season_numeric              28597 non-null  int64         
 8   article_encoded             28597 non-null  int64         
 9   price_article_interaction   28597 non-null  float64       
 10  season_article_interaction  28597 non-null  int64         
 11  weekly_avg_sales            28597 non-null  float64   

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import holidays


from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV, ParameterGrid, TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBRegressor




In [3]:
#   File paths
file_path_EDA = "C:/Users/regi4/OneDrive - Atlantic TU/Year 4/Predictive Analysis Docu's/THE MODEL/French bakery daily sales/Iterations Output/french_bakery_Daily_Sales.csv"
file_path_Feat_Engr = "C:/Users/regi4/OneDrive - Atlantic TU/Year 4/Predictive Analysis Docu's/THE MODEL/French bakery daily sales/Iterations Output/french_bakery_Ft_Eng_1.csv"

iteration_file_path = "C:/Users/regi4/OneDrive - Atlantic TU/Year 4/Predictive Analysis Docu's/THE MODEL/French bakery daily sales/Iterations Output"
results_file_path = "C:/Users/regi4/OneDrive - Atlantic TU/Year 4/Predictive Analysis Docu's/THE MODEL/French bakery daily sales/results"

In [ ]:
data = pd.read_csv(file_path_EDA)
bakery_df = pd.DataFrame(data)

bakery_df['date'] = pd.to_datetime(bakery_df['date'])
bakery_df['Quantity'] = bakery_df['Quantity'].astype(int)
bakery_df['log_Quantity'] = np.log1p(bakery_df['Quantity'])

bakery_df.head()

,date,article,Quantity,unit_price,day_of_the_week,month,season,season_numeric,article_encoded,price_article_interaction,season_article_interaction,weekly_avg_sales,monthly_avg_sales,log_Quantity
0,2021-01-02,BAGUETTE,46,0.90,5,1,Winter,0,3,2.7,0,41.988889,26.250000,3.850148
1,2021-01-02,BANETTE,40,1.05,5,1,Winter,0,6,6.3,0,43.088889,23.326923,3.713572
2,2021-01-02,BANETTINE,6,0.60,5,1,Winter,0,7,4.2,0,5.179775,4.215686,1.945910
3,2021-01-02,BOULE 200G,6,1.10,5,1,Winter,0,10,11.0,0,5.552941,3.340426,1.945910
4,2021-01-02,BOULE 400G,11,1.50,5,1,Winter,0,11,16.5,0,9.277778,4.942308,2.484907


In [ ]:
#   Introducing Interaction Features to the dataset
def create_interaction_features(bakery_ft_df):
    data_int_ft = bakery_ft_df
    
    data_int_ft['log_weekly_avg_sales'] = data_int_ft.groupby(['article_encoded', 'day_of_the_week'])['log_Quantity'].transform('mean')  
    data_int_ft['log_monthly_avg_sales'] = data_int_ft.groupby(['article_encoded', 'month'])['log_Quantity'].transform('mean')

    return data_int_ft

# Function to calculate the SMAPE
#Unlike MAPE, which cannot handle zeros, SMAPE is defined even when actual values are zero.
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return 100 * (abs(y_true - y_pred) / ((abs(y_true) + abs(y_pred)) / 2)).mean()

In [ ]:
#   RandomizedSearchCV for hyperparameter Tuning
def randomized_search_hyperparameters(model, param_grid, X_train, y_train, n_iter):
    #Creating the time-based cross-validator
    timebasedcv = TimeSeriesSplit(n_splits=5)

    randomized_search = RandomizedSearchCV(
        estimator= model,
        param_distributions= param_grid,
        n_iter= n_iter,  #   Number of parameter settings to sample
        scoring = 'neg_mean_absolute_error',
        cv= timebasedcv,
        random_state=42,
        n_jobs=-1
    )

    randomized_search.fit(X_train, y_train)

    return randomized_search.best_estimator_, randomized_search.best_params_

In [ ]:
#   CREATING LAG FEATURES (DAILY, WEEKLY, MONTHLY)
def create_lag_features(bakery_ft_df):
    
    bakery_ft_df = bakery_ft_df.sort_values(by=['article_encoded', 'date'])
    bakery_ft_df.reset_index(drop=True, inplace=True)
    #   Create lag features
    bakery_ft_df['lag_1'] = bakery_ft_df.groupby('article_encoded')['log_Quantity'].shift(1)
    bakery_ft_df['lag_7'] = bakery_ft_df.groupby('article_encoded')['log_Quantity'].shift(7)
    bakery_ft_df['lag_30'] = bakery_ft_df.groupby('article_encoded')['log_Quantity'].shift(30)    
    
    bakery_ft_df[['lag_1', 'lag_7', 'lag_30']] = bakery_ft_df.groupby('article_encoded')[['lag_1', 'lag_7', 'lag_30']].ffill().bfill()
    return bakery_ft_df

#   CREATING MOVING AVERAGES
def create_moving_averages(bakery_ft_df):
    bakery_ft_df = bakery_ft_df.sort_values(by=['article_encoded', 'date']).reset_index(drop=True)

    bakery_ft_df['log_ma_7'] = bakery_ft_df.groupby('article_encoded')['log_Quantity'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
    bakery_ft_df['log_ma_30'] = bakery_ft_df.groupby('article_encoded')['log_Quantity'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())

    return bakery_ft_df

In [ ]:
#   Evaluate XGBoost with transformations (NO REVERSE)

def evaluate_xgboost(model, X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_hold_scaled, y_test, cv):
    xgb_cv_scores = []

    for train_idx, val_idx in cv.split(X_train_scaled):
        X_fold_train, X_fold_val = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        #   Refit Scaler within each fold to avoid leakage
        scaler_fold = StandardScaler()
        X_fold_train = scaler_fold.fit_transform(X_fold_train)
        X_fold_val = scaler_fold.transform(X_fold_val)

        model.fit(X_fold_train, y_fold_train)
        y_fold_pred = model.predict(X_fold_val)
        fold_mae = mean_absolute_error(y_fold_val, y_fold_pred)
        xgb_cv_scores.append(fold_mae)
    
    mean_cv_score = np.mean(xgb_cv_scores)

    #   Fit the model on the full training data
    model.fit(X_train_scaled, y_train)

    #   Predict for the validation and test datasets
    y_validate_pred = model.predict(X_validate_scaled)
    y_test_pred = model.predict(X_test_hold_scaled)

    metrics = {
        "CV Mean MAE": mean_cv_score,
        "Val MAE": mean_absolute_error(y_validate, y_validate_pred),
        "Val MSE": mean_squared_error(y_validate, y_validate_pred),
        "Val R2": r2_score(y_validate, y_validate_pred),
        "Val SMAPE": symmetric_mean_absolute_percentage_error(y_validate, y_validate_pred),
        "Test MAE": mean_absolute_error(y_test, y_test_pred),
        "Test MSE": mean_squared_error(y_test, y_test_pred),
        "Test R2": r2_score(y_test, y_test_pred),
        "Test SMAPE": symmetric_mean_absolute_percentage_error(y_test, y_test_pred)
    }
    return metrics

In [ ]:
#   TO REVERT THE LOG RESULTS TO ORIGINAL SCALE AND THEN TEST THE PERFORMANCE
def evaluate_ORIG_xgboost(model, X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_hold_scaled, y_test, cv):
    xgb_cv_scores = []

    for train_idx, val_idx in cv.split(X_train_scaled):
        X_fold_train, X_fold_val = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        scaler_fold = StandardScaler()
        X_fold_train = scaler_fold.fit_transform(X_fold_train)
        X_fold_val = scaler_fold.transform(X_fold_val)

        model.fit(X_fold_train, y_fold_train)
        y_fold_pred = model.predict(X_fold_val)
        
        # Inverse transform predictions and true values from log scale to original scale
        y_fold_pred_orig = np.expm1(y_fold_pred)
        y_fold_val_orig = np.expm1(y_fold_val)
        
        fold_mae = mean_absolute_error(y_fold_val_orig, y_fold_pred_orig)
        xgb_cv_scores.append(fold_mae)
    
    mean_cv_score = np.mean(xgb_cv_scores)
    model.fit(X_train_scaled, y_train)

    y_validate_pred = model.predict(X_validate_scaled)
    y_test_pred = model.predict(X_test_hold_scaled)
    
    # Convert predictions and targets back to the original scale
    y_validate_pred_orig = np.expm1(y_validate_pred)
    y_validate_orig = np.expm1(y_validate)
    y_test_pred_orig = np.expm1(y_test_pred)
    y_test_orig = np.expm1(y_test)
    
    metrics = {
        "CV Mean MAE (orig)": mean_cv_score,
        "Val MAE (orig)": mean_absolute_error(y_validate_orig, y_validate_pred_orig),
        "Val MSE (orig)": mean_squared_error(y_validate_orig, y_validate_pred_orig),
        "Val R2 (orig)": r2_score(y_validate_orig, y_validate_pred_orig),
        "Val SMAPE (orig)": symmetric_mean_absolute_percentage_error(y_validate_orig, y_validate_pred_orig),
        "Test MAE (orig)": mean_absolute_error(y_test_orig, y_test_pred_orig),
        "Test MSE (orig)": mean_squared_error(y_test_orig, y_test_pred_orig),
        "Test R2 (orig)": r2_score(y_test_orig, y_test_pred_orig),
        "Test SMAPE (orig)": symmetric_mean_absolute_percentage_error(y_test_orig, y_test_pred_orig)
    }
    return metrics


In [ ]:
def evaluate_model(models, features, X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_hold_scaled, y_test, cv):
    results = []
    for model_name, model in models.items():
        metrics = evaluate_ORIG_xgboost(model, X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_hold_scaled, y_test, cv)
        
        metrics["Model"] = model_name
        results.append(metrics)
    
    return pd.DataFrame(results)

In [11]:
xgb_parameter_grid = {
'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 6, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0]
}

bakery_df = create_interaction_features(bakery_df)
bakery_df = create_lag_features(bakery_df)
bakery_df = create_moving_averages(bakery_df)

bakery_df.sort_values(by="date")
bakery_df.reset_index(drop=True, inplace=True)

#   Feature and Target selection with ENCODED FEATURES
features = bakery_df[['day_of_the_week', 'article_encoded', 'unit_price', 'month', 'season_numeric', 'price_article_interaction', 'season_article_interaction', 'log_weekly_avg_sales',
                       'log_monthly_avg_sales', 'lag_1','lag_7','lag_30', 'log_ma_7', 'log_ma_30']]
target = bakery_df['log_Quantity']

bakery_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28597 entries, 0 to 28596
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date                        28597 non-null  datetime64[ns]
 1   article                     28597 non-null  object        
 2   Quantity                    28597 non-null  int64         
 3   unit_price                  28597 non-null  float64       
 4   day_of_the_week             28597 non-null  int64         
 5   month                       28597 non-null  int64         
 6   season                      28597 non-null  object        
 7   season_numeric              28597 non-null  int64         
 8   article_encoded             28597 non-null  int64         
 9   price_article_interaction   28597 non-null  float64       
 10  season_article_interaction  28597 non-null  int64         
 11  weekly_avg_sales            28597 non-null  float64   

In [ ]:
#   TIME SERIES SPLIT FOR EXACTLY ONE YEAR

bakery_df["date"] = pd.to_datetime(bakery_df["date"])
start_date = bakery_df["date"].min()
cutoff_date = start_date+pd.DateOffset(years=1)

training_data = bakery_df[bakery_df["date"] < cutoff_date]
testing_data = bakery_df[bakery_df["date"] >= cutoff_date]

X_train_all = training_data[features.columns]
y_train_all = training_data[target.name]
X_test_hold = testing_data[features.columns]
y_test_hold = testing_data[target.name]

#   Splitting the training data into train and validation sets
val_split_index = int(len(X_train_all) * 0.8)
X_train = X_train_all.iloc[:val_split_index]
y_train = y_train_all.iloc[:val_split_index]
X_validate = X_train_all[val_split_index:]
y_validate = y_train_all.iloc[val_split_index:]

#   Fitting scaler on train set then transforming the val and test with same scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_hold_scaled = scaler.transform(X_test_hold)



In [ ]:
#   Convert the training sets to Dataframe, because X_train_scaled is a numpy array           # SCALING CODE
X_train_scaled = pd.DataFrame(X_train_scaled, columns=features.columns)                              # SCALING CODE
X_validate_scaled = pd.DataFrame(X_validate_scaled, columns=features.columns)                        # SCALING CODE
X_test_hold_scaled = pd.DataFrame(X_test_hold_scaled, columns=features.columns)                      # SCALING CODE

xgb_n_iter = min(20, len(ParameterGrid(xgb_parameter_grid)))
best_xgb, xgb_best_params = randomized_search_hyperparameters(XGBRegressor(), xgb_parameter_grid, X_train, y_train, xgb_n_iter)

# Define model best hyperparameters
models = {
    "XGBoost Regressor": best_xgb
}

timeBased_final_cv = TimeSeriesSplit(n_splits=5)

# Evaluate model
results_df = evaluate_model(models, features, X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_hold_scaled, y_test_hold, timeBased_final_cv)


In [ ]:
# Save results
results_df.to_csv(f"{results_file_path}/2_Daily_ORIG_time_series_split_ONE_YEAR_LOG_QUANTITY_LAG_ALL_MA_features_result.csv", index=False)

In [ ]:
unique_articles = bakery_df['article_encoded'].unique()

from datetime import timedelta

# Get the next day after the end of the dataset
next_day = bakery_df["date"].max() + timedelta(days=1)
day_of_week = next_day.dayofweek
month = next_day.month
season = (month%12 + 3)//3

feature_rows = []

for article in unique_articles:
    # Get recent data for the article
    article_data = bakery_df[bakery_df['article_encoded'] == article].sort_values('date')
    
    if len(article_data) < 30:
        continue 

    last_row = article_data.iloc[-1]
    
    # Generate the lags and moving averages for the future date manually
    recent_sales = article_data['log_Quantity'].values
    lag_1 = recent_sales[-1]
    lag_7 = recent_sales[-7]
    lag_30 = recent_sales[-30]
    
    ma_7 = np.mean(recent_sales[-7:])
    ma_30 = np.mean(recent_sales[-30:])
    
    # Creting the row for next-day prediction
    row = {
        'day_of_the_week': day_of_week,
        'article_encoded': article,
        'unit_price': last_row['unit_price'],
        'month': month,
        'season_numeric': season,
        'price_article_interaction': last_row['unit_price'] * article,
        'season_article_interaction': season * article,
        'log_weekly_avg_sales': last_row['log_weekly_avg_sales'],
        'log_monthly_avg_sales': last_row['log_monthly_avg_sales'],
        'lag_1': lag_1,
        'lag_7': lag_7,
        'lag_30': lag_30,
        'log_ma_7': ma_7,
        'log_ma_30': ma_30
    }
    
    feature_rows.append(row)

next_day_df = pd.DataFrame(feature_rows)

# Scaling using the existing scaler
next_day_scaled = scaler.transform(next_day_df[features.columns])

next_day_log_preds = best_xgb.predict(next_day_scaled)
next_day_preds = np.expm1(next_day_log_preds)

next_day_df['Predicted_Quantity'] = next_day_preds
next_day_df['date'] = next_day

# Undoing the label encoding on the articles to get their name
next_day_df['article'] = label_encoder_article.inverse_transform(next_day_df['article_encoded'])

next_day_df[['date', 'article', 'Predicted_Quantity']].to_csv(f"{results_file_path}/Tomorrow_Predictions.csv", index=False)

In [ ]:
#   SEND PREDICTIONS/FILE TO S3 BUCKET
import logging
import boto3
from botocore.exceptions import ClientError

def upload_file(file_path, bucket, object_name):
    s3_client = boto3.client('s3',
                             aws_access_key_id="aws-access-key-id",
                             aws_secret_access_key="aws-secret-access-key",
                             region_name="eu-west-1")
    try:
        response = s3_client.upload_file(file_path, bucket, object_name)
        print(f"Upload successful: {file_path} → s3://{bucket}/{object_name}")
        return True
    except ClientError as e:
        logging.error(e)
        print(f"Upload failed: {e}")
        return False

output_file = "Tomorrow_Predictions.csv"
s3bucket = "test-pred-bucket"
file_path = f"{results_file_path}/{output_file}"
object_name = "Predictions.csv"
upload_success = upload_file(file_path, s3bucket, object_name)

Upload successful: C:/Users/regi4/OneDrive - Atlantic TU/Year 4/Predictive Analysis Docu's/THE MODEL/French bakery daily sales/results/Tomorrow_Predictions.csv → s3://test-pred-bucket/Predictions.csv
